# Divide and Conquer 

* [O(n log n) Algorithm for Counting Inversions](#inv)
    * [Inversions](#inv1)
    * [Divide and Conquer](#inv2)
    * [Count Split Inversions Subroutine and Merge Sort](#inv3)
    * [Analysis](#inv4)
    * [Solution: Counting Split Inversions in Linear Time](#inv5)
* [Strassen's Subcubic Matrix Multiplication Algorithm](#mat)
    * [Applying Divide and Conquer](#mat1)
    * [Recursive Algorithm #1](#mat2)
    * [Strassen's Algorithm (1969)](#mat3)
* [O(n log n) Algorithm for Closest Pairs](#pair)
    * [Pseudocode](#pair1)
    * [ClosestSplitPair(P<sub>x</sub>P<sub>y</sub>) Pseudocode](#pair2)
* [Q&A](#qa)

## Main Idea
1. Divide the problem into smaller subproblems (sometimes conceptually,  sometimes literally)
2. Conquer the subproblems via recursion
3. Combine subproblem solutions into one for the original problem

## O(n log n) Algorithm for Counting Inversions <a class="anchor" id ="inv"></a>

* __Inpuut__: Array A containing the numbers 1, 2, 3, ... in some arbitrary order
* __Output__: number of inversions --> number of pairs (i, j) of array indices with i < j and A[i] > A[j]
    * ie: a fully sorted series of numbers would not have any inversions, because all items are less than those that follow it

### Inversions <a class="anchor" id ="inv1"></a>

* A = (1, 3, 5 ,2, 4, 6)
* Inversions: 
    * (3, 2) --> (A[2], A[4])
    * (5, 2) --> (A[3], A[4])
    * (5, 4) --> (A[3], A[5])
<img src ="resources/inversions.PNG">  
* Motivation - Numerical similarity measure that quantifies how close 2 lists are to each other 
    * In general, the more inversions a list has, the more different it would be 
* Collaborative filtering - process of filtering for information or patterns using techniques involving collaboration among multiple agents, viewpoints, data sources, etc.
    * Method of making automatic predictions about the interests of a user by collecting preferences or taste information from many users
* ___In an array of size n, the largest number of inversions is 'N choose 2' or (n(n-1)/2)___
    * The worst case is when the array is in backwards order and every single pair is inverted 
    * _N choose 2 evaluates to a quadratic expression_ 
* __Brute-force__ - set up a double for loop, one which goes through i and one which goes through j > i, checking each pair ij individually for inversion, and then add it to our running count. Return final count at the end --> __O(n<sup>2</sup>) time__ 

### Divide and Conquer <a class="anchor" id ="inv2"></a>

* Classify inversions into 3 types
    1. Left inversion - both array indices in the inversion pair <= n/2
        * Fully counted by recursing on the left half of an array
    2. Right inversion - both array indices in the inversion pair > n/2
        * Fully counted by recursing on the right half of an array
    3. Split inversion - smaller index is <= n/2 and the larger one is > n/2
        * Needs its own subroutine/cleanup after recursion
* __General Concept__: 
    * Function Count(array A, length n):
        1. Base case: if n = 1, return 0
        2. x = count(1st half of A, n/2)
        3. y = count(2nd half of A, n/2)
        4. z = countSplitInv(A, n)
        5. Return x + y + z
* __Goal__ - implement CountSplitInv in linear(O(n)) time, to making the total process run in O(n log n)
    * Just like merge sort - 2 recursive calls on a problem of n/2 and outside of those two calls, only linear work

### Count Split Inversions Subroutine + Merge Sort <a class="anchor" id ="inv3"></a>

* Worst case, there could be a quadratic number of split inversions that we want to count in linear time
* Piggy-back on merge sort - Merge subroutine will actually naturally uncover split inversions _in linear time_
    * Implement a sorting algorithm in our recursive calls
* __Better Plan for Counting Inversions__: 
    * Function sortCount(A, n):
        1. Same base case: if n = 1, return 0
        2. B, x = sortCount(first half of A, n/2) 
        3. C, y = sortCount(second half of A, n/2) 
        4. D, z = mergeCountSplitInv(A, n) 
        5. return x + y + z 
<hr>
    * Merge Pseudocode
        * D = output [length = n]
        * B = 1st sorted array [n/2]
        * C = 2nd sorted array[n/2]
        * i = 1 --> pointer for B
        * j = 1 --> pointer for C
        * k --> index for D
        ```
            for k = 1 to n:
                if B(i) < C(j)
                    D(k) = B(i)
                    i++
                else [C(j) < B(i)]
                    D(k) = c(j)
                    j++
             end
           ```
<hr>
* __Array with no split inversions__
    * Because all elements in B will be less than all elements in C, the merge is unusually quick - it just copies B into D and then concatenates C
    * _This suggests that copying elements from the second subarray C has something to do with the number of split inversions in the original array_
* __Array with _only_ split inversions__
    * A = (1, 3, 5, 2, 4, 6)
    * B = (1, 3, 5) and C = (2, 4, 6)
    * D = empty output array
    1. Copy the first item from B (1) to D, i ++
    2. Copy the first element from C (2), which is next in sort order
        * i++
        * When an item from the second (right) array is copied before the first (left) array has been exhausted, any items remaining in the left array indicate inversions. Here, they are (3, 2) and (5, 2)
    3. Copy the next item in sort order - it's from the first array, i ++
    4. Copy the next item in sort order - it's from the second array
        * There is still an item in the left array, so another inversion is exposed (5, 4)
    5. Continue/End  

### Analysis <a class="anchor" id ="inv4"></a>

* __General Claim__: The split inversions that involve an element y of the second array C are precisely those elements remaining in the first array B when y is copied over to the output array. 
* __Proof__: Let x be an element of the first array B
    1. If x is copied to output before y, then x < y => there is no inversion involving x and y
    2. If y is copied over before x, then y < x. By nature of being in right array, y's index _must_ be greater than that of the item it precedes => split inversion involving x (and any other elements remaining in B) and y

### Solution: Counting Split Inversions in Linear Time <a class="anchor" id ="inv5"></a>

* Really just augmented merge sort code
    * When an element of C gets copied to output D, increment the running count of split inversions by the number of elements remaining in B (n/2 - i)
* __Pseudocode__: 
    * D = output [length = n]
    * B = 1st sorted array [n/2]
    * C = 2nd sorted array[n/2]
    * i = 1 --> pointer for B
    * j = 1 --> pointer for C
    * k --> index for D
    * count --> running count of split inversions
     ```
        for k = 1 to n:
            if B(i) < C(j)
                D(k) = B(i)
                i++
            else [C(j) < B(i)]
                D(k) = C(j)
                count += (n/2) - i
                j++             
        end
       ```
* __Running Time__:
    * Split Inversions Subroutine
        * Merge subroutine is known to be linear time
        * The only additional step here is the incrementation of the counter, which is constant time
            * Constant C1 so that the merge step takes at most C1n steps
            * Constants C2 so that the rest of the work is in those C2n steps
            * added together we get at most (C1 + C2)n steps --> still O(n) because C1/2 are constants
        * Results in O(n) + O(n) = O(n) (as long as n!= 2)
    * Overall
        * 2 recursive calls on n/2 --> log n
        * linear work outside of the recursive calls --> O(n)
        * Results in O(n log n)

## Strassen's Subcubic Matrix Multiplication Algorithm <a class="anchor" id ="mat"></a>

* __Problem__: How do you multiply 2 n by n (square) matrices X and Y, producing a new n by n  (square) matrix Z?
    * where Z(ij) = dot product of the i row of x and j column of Y
    * _Note: normally, n denotes input size, but here is denotes the dimensions of the matrices. Each matrix is n by n, containing n<sup>2</sup> entries_
    <img src="resources/matrix_multi_formula.PNG">
    * Because we have to read in an input of n<sup>2</sup> and produce an output of n<sup>2</sup>, the best case is n<sup>2</sup> time
    * Example:
    <img src= "resources/matrix_multi_ex.PNG">
* Iterative method for obtaining the dot product is O(n<sup>3</sup>)
    * Triple for loop which computes each entry of the output array separately using the dot product
    * For each iteration of the outer loop, the total number of runs in the inner loops would be equivalent to n, so you end up with n * n * n  or O(n<sup>3</sup>)

### Applying Divide and Conquer  <a class="anchor" id ="mat1"></a>

* __General Idea__:
    1. Divide square matrices X and Y into quadrants/blocks
        * Where A-H are all n/2 by n/2 matrices
    2. Dot Product using the quadrant names
       <img src ="resources/dotprod.PNG">
* This is essentially the same thing we did in the integer multiplication exercise --> break it into smaller pieces, take the expansions, and observe how that expansion could be written in terms of n/2 digit numbers.

### Recursive Algorithm #1 <a class="anchor" id ="mat2"></a>

1. Recursively compute the 8 necessary products (ae, bg, af, bh, ce, dg, cf, dh)
2. do the necessary additions (4) --> O(n<sup>2</sup>)
* __Fun Fact__: In a recursive algorithm where you do 8 recursive calls each on a problem withdimensions half as much as what you started with and then do quadratic time outside, the time will be cubic.

### Strassen's Algorithm (1969) <a class="anchor" id ="mat3"></a>

1. Recursively compute only _7_ (cleverly chosen) products
2. Do the necessary (clever) additions and subtractions --> still O(n<sup></sup>) at this step
* __Fun Fact__: Shaving off one recursive call changes this from a cubic process to SUBCUBIC! (see Master Method notes)

#### The Details

<img src = "resources/dotprod.PNG">

* The 7 products:
    1. p1 = a(f - h)
    2. p2 = (a + b)h
    3. p3 = (c + d)e
    4. p4 = d(g - e)
    5. p5 = (a + d)(e + h)
    6. p6 = (b - d)(g + h)
    7. p7 = (a - c)(e + f)
* __Claim__: The ouput matrix Z from X * Y can be found using these 7 products in the following manner, separated by quadrants:
p5 + p4 - P2 + p6 || p1 + p2
p3 + p4           || p1 + p5 - p3 - p7
* __Proof__: Just looking at upper left corner (ae + bg)
    1. Expand the relevant products (parenthesis n/a to arithmetic)
        * (ae + ah + de + dh) + (dg - de) - (ha + hb) + (bg + bh - dg - dh)
    2. Look for cancellations
        * +/- de
        * +/- dh
        * +/- dg
        * +/- ah
        * +/- bh
    3. Only ae +  bg remaining, which is exactly what is in our Top LH corner

## O(n log n) Algorithm for Closest Pair <a class="anchor" id ="pair"></a>

* __Input__: a set p = {p<sub>1</sub>, ..., p<sub>n</sub>} of n points in the plane 
* __Notation__: We are talking about the _Euclidian Distance_
    <img src="resources/dist.PNG">
* __Output__: a pair p*, q* in P that minimizes d(p,q) over p,q in the set P
* __Initial Observations__: 
    * Assume all points have distinct x and y coord (for convenience)
    * Just like in the inversions problem, this can be solved by brute-force with nested for loops (O(n<sup>2</sup>))
    * 1-D version (a line)
        * Sort points (O(n log n))
        * Return closest pair of adjacent points (o(n))

#### High Level Approach 

1. Make 2 copies of the points for sorting
    * (Px) - set of points sorted by x-coord
    * (Py) - set of points sorted by y-coord
    * O(n log n)
<img src="resources/pair.PNG">
2. Divide and Conquer

### Pseudocode <a class="anchor" id="pair1"></a>

* ClosestPair(P<sub>x</sub>,P<sub>y</sub>)
    * Omit the base case but it would be essentially brute force at a very small n
    1. _let Q = left half of P, R = right half of P_ --> (O(n log n))
        * Form Qx, Qy, Rx, Ry by linear scan --> Copies of Q, R sorted by x and y 
    2. Recursive calls --> O(log n)
        * `(p1, q1) = ClosestPair(Qx, Qy)`  
        * `(p2, q2) = ClosestPair(Rx, Ry)`  
    3. ClosestSplitPair Subroutine --> Goal is O(n) to keep us at O(n log n)
        * If the closest pair is split between the two sides, then the recursive calls won't find it
        * `(p3, q3) = ClosestSplitPair(Px, Py)`
    4. Return best of (p1, q1), (p2, q2), (p3, q3)

* __Key Idea__ - Only need to bother computing the closest split pair in the "unlucky case" where its distance is less than the result of the first and second recursive calls
* __Better Option__: Keep steps 1 and 2
* ClosestPair(P<sub>x</sub>,P<sub>y</sub>)
    1. _let Q = left half of P, R = right half of P_ --> (O(n log n))
        * Form Qx, Qy, Rx, Ry by linear scan --> Copies of Q, R sorted by x and y 
    2. Recursive calls --> O(log n)
        * `(p1, q1) = ClosestPair(Qx, Qy)`  
        * `(p2, q2) = ClosestPair(Rx, Ry)` 
    3. Let `delta =  min{d(p1, q1), d(p2, q2)}`
    4. `(p3, q3) = ClosestSplitPair(Px, Py, delta)`
        * returns Null if no split pairs, or the new best pair if exists
    5. Return best of (p<sub>1</sub>, q<sub>1</sub>), (p<sub>2</sub>, q<sub>2</sub>), (p<sub>3</sub>, q<sub>3</sub>)

### ClosestSplitPair(P<sub>x</sub>, P<sub>y</sub>, delta) Pseudocode <a class="anchor" id="pair2"></a>

* Correctness measures:
    1. O(n) time
    2. Correct whenever closest pair of P is a split pair
* Delta is the parameter that controls whether or not we  actually care about the closest split pair, because we only care if there  is a split pair at distance < delta.
* Prune away extra points and focus on those that lie in a vertical strip of width 2(delta) which is roughly centered in the middle of the point set. For the purposes of this subroutine, the points in this strip are the only ones we care about anymore

<img src = "resources/split_pair.PNG">

* ClosestSplitPair(P<sub>x</sub>, P<sub>y</sub>, delta):
    * _Let xbar = biggest x-coordinate in left of P_ --> O(1) time
    * _Let Sy = points of P with x-coordinate in the interval [(xbar - delta), (xbar + delta)] sorted by y coordinate_
    
    * Initialize a variable to keep track of best candidate and best pair 'so far'  
        * `best = delta`
        * `best_pair = Null` 
        
`for i = 1 to |Sy| - 7:
    for j = 1 to 7:
        let p,q = i, (i+j) points of Sy
        if d(p,q) < best
            best pair = (p,q)
            best = d(p,q) `
 
* __Running Time__:
    * O(1) time for initializing variables
    * O(n) for outer loop 
    * O(1) for the inner loop - we only look at a constant number of other positions
    * __Subroutine Total__ - O(n)
    * __Algo Total__: O(n log n)
        * O(n log n) to sort and split lists
        * O(log n) for recursive calls
        * O(n) for split pairs 


#### Correctness Claim 

* __Correctness Claim__: Let there exist a P<sub>q</sub>, Q<sub>r</sub> be a split pair with distance (d(pq)) < delta, THEN:
    * _A - p and q are members of S<sub>y</sub>_
    * _B - p and q are at most 7 positions apart ins S<sub>y</sub>_
    * If this is true then:
        * __Corollary1__: If the closest pair of P is a split pair, then ClosestSplitPair finds it
        * __Corollary2__: ClosestPair is correct and runs in O(n log(n)) time


#### Proof of Claim A

* __Proof of Correctness Claim (A)__: 
    * let p = (x1, y1) from the left half of the point set (Q), and we have point q = (x2,y2) from the right side of the point set (R), d(p,q) < delta
    * _Note: Since d(p,q) < delta, |x1-x2| < delta and |y1-y2| < delta_
    * A is really saying that p and q are within delta of Xbar 
    <img src="resources/pairs_proofA.PNG">
    * Xbar = the n/2th X-coord from the left most set (ie: the farthest right)
    * p distance:
        1. p is from the left half of the plane
        2. We are assuming that there is a split pair that has distance less than delta
        3. p can only be _at most_ xbar, the farthest right coord in the left set --> p <= xbar
    * q distance:
        1. q is from the right half of the plane
        2. xbar is the farthest right from the _left_ half of the plane
        3. q _must_ be at least xbar --> q > xbar
    * d(p,q):
        1. We are assuming there is a split pair that has distance less than delta
        2. Because p can at most be xbar, q can at most be xbar + delta 
        3. Because q must be at least xbar, p can be at minimum xbar - delta
    * Sy
        1. Sy is defined as the vertical strip of the plane with bounds xbar +- delta
        2. xbar - delta <= p <= xbar <= q <= xbar + delta
        3. So p and q must be inside the strip. 

#### Proof of Claim B

* __Proof of Correctness Claim (B)__: 
    * p(x1, y1) and q (x2, y2) are at most 7 partitions apart in Sy
    * Imagine delta/2 x delta/2 boxes with center xbar and min{y1, y2}
    <img src = "resources/pairs_proofB.PNG">
    * Lemma 1: All points of Sy with y-coordinate between those of p and q inclusive, lie in one of these 8 boxes
        1. If the Euclidean distance is less than delta, the Y coordinates of p, q must differ by <= delta
        2. The point with the greater y (in this case, p) can only be <= y(min) + delta from the bottom at the farthest
        3. By definition of Sy, all points inside it have x-coordinates between xbar +- delta
    * Lemma 2: There can only be 0 or 1 point in each of the 8 boxes
        * This is also why we only had to worry about split pairs in the "unlucky" case
        * _Proof by Contradiction_ - Suppose a, b lie in the same box at antipodal corners, THEN
        <img src = "resources/pairs_proofB2.PNG">
        1. a, b are either both in Q or both in R 
            * Each box is entirely in either Q  or R
            * Any point with x coordinate at most xbar must be in Q, any x-coordinate at least xbar must be in R
            * Here, a and b both have x-coordinate > xbar, so they are both in R
        2. If a,b share a box, d(a, b) can at most be (delta/2)(root(2))
            * Pythagorean theorem for that expression
            * delta = d(closest pair) on the same side, as calculated by our recursive calls and passed into the SplitPairs subroutine
            * By extension of delta's definition, no two points in the same set Q, R can have d < delta
            * ___(delta/2)(root(2)) < delta___
        * These two consequences in tandem contradicts the definition of delta

#### Wrap-up

* Lemma 1 proves every relevant point that survives the filtering and makes it into Sy has to be in one of these 8 boxes. Lemma 2 proves that there can be at most one point in each box, or 8 points total, including p and q.
* If p is our first point, and q is our last, they can at most be 7 apart (8-1 = 7)
* This means that in our algorithm, our double for loop (limited at 7) is guaranteed to identify any split pair with d(p, q) < delta

## Quiz Questions <a class="anchor" id ="qa"></a>

* What is the largest number of inversions that a 6-element array can have? 
    * 15
<br>

* Given an array A where all inversions are either left or right (no split inversions). What is the relationship between the sorted subarrays B and C?
    * All elements of B are less than all elements of C
<br>

* What is the running time of the straightforward iterative algorithm for matrix multiplication?
    * O(n<sup>3</sup>)
<br>

* Suppose we can correctly implement the ClosestSplitPair subroutine in _O(n)_ time. What will be the overall running time of the Closest Pair algorithm? 
    * O(n log n)